<img src="https://www.strath.ac.uk/media/1newwebsite/webteam/logos/xUoS_Logo_Horizontal.png.pagespeed.ic.M6gv_BmDx1.png" width="350" align="left" style="left">

## Automatic Gain Control with PYNQ

We present a digital Automatic Gain Control (AGC) circuit with interactive control of it's parameters. This is a purely digital loopback system, so no extra hardware is required. We'll generate various input signals, explore some interesting effects of the AGC algorithm, and practice tweaking our parameters for best performance.
The design of this AGC example will be explored, featuring various hardware arithmetic approximations for power estimation, logarithms and exponentiation.

## Aims
* Interactively explore the behaviour of an AGC circuit
* Learn to tune the AGC parameters for different scenarios
* Explore our open-source hardware design, with a focus on arithmetic approximations

## Table of Contents

* [1. Introduction](#1.-Introduction)
* [2. ...And we're off!](#go)
* [3. Exploring some scenarios](#scenarios)
* [4. A closer look at the hardware](#hw)
    * [4.1 Power estimation](#power-est)
    * [4.2 Logarithms and exponentials](#log-est)
* [5. Conclusion](#conclusion)

----

## 1. Introduction

This example is a purely digital loopback, so there is no requirement for extra hardware. If you've got a Pynq Z2 board, ZCU111, or RFSoC 2x2, you can run it and gave a play with all of the interactive features. Let's take a look at how this demonstration is structured.

<a class="anchor" id="fig-1"></a>
<figure>
    <img src='./assets/dagc_loopback_overview.svg'/>
    <figcaption><b>Figure 1: System overview </b></figcaption>
</figure>

The datapath is a simple loopback and we'll generally be following only a few steps:

  * Generate a signal in software
  * Stream our input signal _to_ the AGC circuit, via the first DMA
  * Stream our output signal _from_ the AGC, via the second DMA
  * Inspect the output signal via this notebook

We should also note that there are a few parameters for the AGC circuit, which we can set from software as well. Just before we start interacting with our AGC circuit, let's take a moment to look at it's design and appreciate what its behaviour should be. Our circuit is loosely based on the algorithm presented in [this book](https://www.elsevier.com/books/rf-and-digital-signal-processing-for-software-defined-radio/rouphael/978-0-7506-8210-7)[1] (and there's a free excerpt from the AGC section [online](https://www.edn.com/wireless-101-automatic-gain-control-agc/)[2]).

### Why use AGC?

The main goal of the AGC circuit is to keep a signal's average power steady, at a known level. This is used to mitigate fluctuations in a signal's received power. In the analogue world, this is often to ensure that we present a full-scale signal to our ADC to minimise the impact of quantisation errors. We'll show the use of such an external analogue gain with RFSoC in a future notebook. However, there are quite a few uses in the digital world too --- generally when we want to normalise signals digitally at IF or baseband. Synchronisation for QAM receivers is one example. The synchronisation task is much simpler if we have a priori knowledge of the signal's average power, since we know what amplitude to expect for each point in the constellation. We can use an AGC, just like this one, to ensure our synchronisation circuit is given a signal with it's preferred average power. This can not only correct for differences in the received signal's power, but also any changes over time.

### Our Design

Our AGC circuit is a feedback loop that will amplify our signal, aiming to match a reference average power. There are 3 main parameters we will be tuning from software: the desired average power (ref), the number of samples used to calculate the average power (window), and how aggressively we want to vary our gain (alpha).

<a class="anchor" id="fig-1"></a>
<figure>
    <img src='./assets/dagc_block_diagram.svg'/>
    <figcaption><b>Figure 2: AGC block diagram</b></figcaption>
</figure>

Our control is based  gain based on the _logarithm_ of the input power... and these $log_{10}(x)$ and $10^{x}$ blocks _really_ complicate the digital design. A much simpler AGC design could respond linearly (instead of logarithmically) but this would have slightly different behaviour. We opt for the more painful route for two reasons:

  * The log response lets us react to large abrupt changes to the input signal roughly as quickly as we can to small changes. This is a nice property to have when dealing with packets or bursts of data. It also simplifies any analysis if we can assume that the "recovery time" is independent of the input signal.
  
  * It will make it easier to interface to analogue Variable Gain Amplpifiers (VGAs) in the future. Most of VGAs offer a control for their gain that is "linear in dB" (i.e. it's logarithmic!). By having our control loop already using logarithms, we could have direct linear control of a VGA without extra interfacing circuitry.

Let's now explore the behaviour of our system while getting our hands dirty

## 2. ...And we're off! <a class="anchor" id="go"></a>

Run the following code cell to launch our GUI. This let's you set AGC parameters and generate your own input signals. The input signals will automatically be streamed through the AGC circuit and visualised below. You can control the envelope of the input signal by clicking and dragging the "envelope handles" (those blue circles). Note that the window parameter (how many samples are averaged during our power calculation) is shown on the output plot as the grey area at the far left.

Check [this animation](https://github.com/strath-sdr/pynq_agc/blob/master/demonstration.gif) to see an example of setting the envelope.

In [ ]:
import pynq_agc

model = pynq_agc.AgcDashModel(fs=int(1e6), N=6000)
app = pynq_agc.AgcDashController(model)
app.show()

We'd recommend having a play with different envelopes, signal types, and AGC parameters to get an intuition for how the AGC behaves. Once you've done some exploring, we present a couple of (somewhat) realistic scenarios and discuss them below.

## 3. Exploring some scenarios

Here are a few examples of configurations which demonstrate real-world communications challenges related to AGC.
You can select each scenario the with "Preset Scenarios" input in the GUI.

> ### Slow Fading
> Here we have an effect that emulates "slow fading" of a simple signal. We'd usually expect this to be caused by movement, often with buildings or terrain impeding the signal. If the receiver is sensitive to the amplitude of the received signal (e.g. AM or many QAM synchronisers), we'd need use an AGC to compensate for the changes in signal strength.
>
> Try different window and alpha parameters to see how constant you can keep the output power. Notice that increasing the window size will reduce the time resolution of our gain control, which leads to a noticable saw-tooth envelope for slow fading effects. If alpha is too high, we can see the output ringing where we've overshot by adjusting the gain too agressively. If alpha is too low, we might not be compensating quickly enough for the fading effects.

> ### AM Envelope Preservation
> AM demodulation is a classic AGC example. For AM radio receivers, we want to play baseband signals with a fixed average power so broadcasts from different locations are all audible (and more powerful ones aren't deafening!). Our information is encoded in the envelope of the signal and we need to be very careful not to distort it. If our AGC reacts too quickly, it can start to smooth out the envelope, destroying our information. Try reducing the window until this starts to happen. The window length is shown as the gray rectangle on the output plot. Try to achieve the same effect by decreasing the modulated "Data" frequency as well.

> ### Packet Preambles with QPSK
> Here we present a bursty QPSK transmission, as if receiving a single packet.
> We can clearly see the recovery time the AGC needs in order to converge to a steady gain. Having this steady, known power can be important for synchronisation (especially for higher order QAMs) and we can change this level using the ref parameter. However the AGC's recovery time means that the first few symbols will likely not be received correctly. In this situation we need to tune our AGC parameters to ensure we'll reach equilibrium before the packet's preamble is finished (or vice versa). We can tune our alpha, ref, and window parameters and then deduce the minimum preamble length that would be approapriate for this packet format. Just like our AM scenario, QPSK and QAM signals will start to become destroyed if our window is small relative to the symbol period.

## 4. A closer look at the hardware <a class="anchor" id="hw"></a>

This section will give some insights into the FPGA design for our AGC and we'll focus on the fixed-point arithmetic techniques used to implement the more exotic functions such as power estimation, logarithms, exponentiation. First of all, let's revisit our block diagram.

<a class="anchor" id="fig-3"></a>
<figure>
    <img src='./assets/dagc_block_diagram.svg'/>
    <figcaption><b>Figure 3: Revisiting the AGC block diagram</b></figcaption>
</figure>

This model presents quite a concise mathematical system. Drawing from the loop analysis presented by [Tony Rouphael](https://www.elsevier.com/books/rf-and-digital-signal-processing-for-software-defined-radio/rouphael/978-0-7506-8210-7), we can formally represent this model with a few managable equations. 

Our averaged input power, $P_{in}$, is at the output of our integrate-and-dump block. Given inputs $I_b(n)$, $Q_b(n)$, and our window parameter $N$, we define $P_{in}(n)$ as: 

$$ P_{in}(n) = \frac{1}{N} \sum_{k=0}^{N-1}\sqrt{ I_b^2(n-k) + Q_b^2(n-k) } $$

We also need to find an equation for the output of our accumulator, $v(n)$. Given our alpha and ref parameters, $\alpha$ and $P_{ref}$, we can write an expression for $v(n)$ in state-space representation:

$$ v(n+1) = v(n) + \alpha [P_{ref} - \log_{10}(10^{v(n)} P_{in}(n))] $$

And finally to close the loop, we can model the outputs of our gain stage, $I_b(n)$ and $Q_b(n)$, in terms of the input signal, $I_a(n)$ and $Q_a(n)$, and $v(n)$.

$$ I_b(n) = 10^{v(n)}I_a(n) \\ Q_b(n) = 10^{v(n)}Q_a(n) $$

Although the _analysis_ of this system can be quite complex, it is at least simple to describe using this state-space form. In fact, we could translate these equations to a software implementation with very little effort.

In [ ]:
import numpy as np

def python_agc(i_a, q_a, p_ref, alpha, N):
    
    v = 0
    i_b = np.zeros((len(i_a),))
    q_b = np.zeros((len(q_a),))
    
    for (n,(i,q)) in enumerate(zip(i_a,q_a)):
            i_b[n] = (10**v * i)
            q_b[n] = (10**v * q)
    
            # Protect against averaging empty lists           vvvvvvvvvvvvvv
            p_in = np.average( np.abs(i_b + 1j*q_b)[n-N:n]    if n>=N else 0 )
              
            # Protect against log10(0) for later              vvvvvvvvvvvvvvvvvvvvvv
            inner_term = 10**v * p_in                         if p_in > 0 else 1e-12
                
            v = v + alpha * (p_ref - np.log10(inner_term))
            
    return (i_b, q_b)

In [ ]:
(i_a, q_a) = model.test_input( model.ref_signal('sin', 80000, 2000),
                               model.envelope([(0.003,1.0), (0.0031,0.5)]) ) 

(i_b, q_b) = python_agc(i_a, q_a, np.log10(0.2), 0.8, 16)

In [ ]:
import plotly.express as px
import pandas as pd
px.line(pd.DataFrame({'x':range(len(i_a[1000:])), 'y':q_b[1000:]}), 'x', 'y')

So, if a software implementation (albeit a _very_ slow one) is so easy to implement, what is the challenge in translating this to hardware? Of course the standard answers apply here, such as complexity introduced by implementing AXI-Stream / DataFlow control, fixed-point conversion, pipelining, etc. In this particular case, though, there are two interesting challenges: power estimation and logarithms/exponentials.

### 4.1. Power estimation <a class="anchor" id="power-est"></a>
### 4.2. Logarithms and exponentials <a class="anchor" id="log-est"></a>

## 5. Conclusion <a class="anchor" id="conclusion"></a>

Oh what fun we've had.


## References <a class="anchor" id="references"></a>

[1] - [Tony Rouphael, "RF and Digital Signal Processing for Software-Defined Radio"](https://www.elsevier.com/books/rf-and-digital-signal-processing-for-software-defined-radio/rouphael/978-0-7506-8210-7)

[2] - [EDN, "Wireless 101: Automatic Gain Control (AGC)", Accessed: 09/02/2021](https://www.edn.com/wireless-101-automatic-gain-control-agc/)

[3] - [Dana Whitlow, Analog Devices, Design and Operation of Automatic Gain Control Loops for Receivers in Modern Communications Systems](https://www.analog.com/media/en/training-seminars/tutorials/42575412022953450461111812375Design_and_Operation_of_AGC_Loops.pdf)

[3] - [Grant Griffin, DSPGuru, "DSP Trick: Magnitude Estimator", Accessed: 09/02/2021](https://dspguru.com/dsp/tricks/magnitude-estimator/)

[4] - [Yevgen Voronenko, SPIRAL Project, "Multiplier Block Generator", Accessed: 09/02/2021](http://spiral.ece.cmu.edu/mcm/gen.html)

## Revision History
* **v0.1** | *Craig Ramsay* | 09/02/2021 | First alpha demo

[⬅️ Previous Notebook](previous_notebook.ipynb) | | [Next Notebook ➡️](next_notebook.ipynb)

----
----